In [1]:
# Install necessary libraries
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.3.1 transformers==4.41.3 datasets==2.20.0 accelerate==0.31.0 --progress-bar off
!pip install -qqq bitsandbytes==0.43.1 peft==0.11.1 trl==0.9.4 openpyxl --progress-bar off  # For CSV and xlsx support


ERROR: Ignored the following yanked versions: 4.14.0, 4.25.0, 4.46.0
ERROR: Could not find a version that satisfies the requirement transformers==4.41.3 (from versions: 0.1, 2.0.0, 2.1.0, 2.1.1, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.8.0, 2.9.0, 2.9.1, 2.10.0, 2.11.0, 3.0.0, 3.0.1, 3.0.2, 3.1.0, 3.2.0, 3.3.0, 3.3.1, 3.4.0, 3.5.0, 3.5.1, 4.0.0rc1, 4.0.0, 4.0.1, 4.1.0, 4.1.1, 4.2.0, 4.2.1, 4.2.2, 4.3.0rc1, 4.3.0, 4.3.1, 4.3.2, 4.3.3, 4.4.0, 4.4.1, 4.4.2, 4.5.0, 4.5.1, 4.6.0, 4.6.1, 4.7.0, 4.8.0, 4.8.1, 4.8.2, 4.9.0, 4.9.1, 4.9.2, 4.10.0, 4.10.1, 4.10.2, 4.10.3, 4.11.0, 4.11.1, 4.11.2, 4.11.3, 4.12.0, 4.12.1, 4.12.2, 4.12.3, 4.12.4, 4.12.5, 4.13.0, 4.14.1, 4.15.0, 4.16.0, 4.16.1, 4.16.2, 4.17.0, 4.18.0, 4.19.0, 4.19.1, 4.19.2, 4.19.3, 4.19.4, 4.20.0, 4.20.1, 4.21.0, 4.21.1, 4.21.2, 4.21.3, 4.22.0, 4.22.1, 4.22.2, 4.23.0, 4.23.1, 4.24.0, 4.25.1, 4.26.0, 4.26.1, 4.27.0, 4.27.1, 4.27.2, 4.27.3, 4.27.4, 4.28.0, 4.28.1, 4.29.0, 4.29.1, 4.29.2, 4.30.0, 4.30.1, 4

In [2]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")



Using device: cuda


In [3]:
import pandas as pd
from google.colab import files

# Upload the questionanswer.csv file
uploaded = files.upload()  # Select the questionanswer.csv file

# Load the dataset and check columns
df = pd.read_csv("questionanswer.csv", encoding='ISO-8859-1')
print(df.head())

# Ensure columns are as expected
assert set(df.columns) == {"Question", "Answer", "Type"}, "Columns must be: Question, Answer, Type"


Saving questionanswer.csv to questionanswer.csv
                                            Question  \
0  About how many years ago did life originate on...   
1                        Example for Endemic species   
2            write a short notes lectron microscopes   
3         Explain the fine structure of mitochondria   
4  Indicate the major function of both vitamin K ...   

                                              Answer       Type  
0                                           3.5X10^9  Structure  
1  Dipterocurpus zeylanicus / Gracinia quaesita\n...  Structure  
2  Electron microscopes represent a groundbreakin...      Essay  
3  Fine Structure of Mitochondria\n\nMitochondria...      Essay  
4                                Aids blood clotting  Structure  


In [4]:
from huggingface_hub import login

# Replace 'your_hf_token' with the actual token
login(token="")


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"
PAD_TOKEN = "<|pad|>"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load tokenizer with padding
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, use_auth_token=True)
tokenizer.add_special_tokens({"pad_token": PAD_TOKEN})
tokenizer.padding_side = "right"

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    device_map="auto",
    use_auth_token=True
)
model.resize_token_embeddings(len(tokenizer))


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Embedding(128257, 4096)

In [6]:
# Create formatted text with Type, Question, and Answer
def format_example(row):
    return f"Type: {row['Type']}\nQuestion: {row['Question']}\nAnswer: {row['Answer']}"

df['formatted'] = df.apply(format_example, axis=1)
print(df['formatted'].head())  # Verify formatting


0    Type: Structure\nQuestion: About how many year...
1    Type: Structure\nQuestion: Example for Endemic...
2    Type: Essay\nQuestion: write a short notes lec...
3    Type: Essay\nQuestion: Explain the fine struct...
4    Type: Structure\nQuestion: Indicate the major ...
Name: formatted, dtype: object


In [7]:
# Convert DataFrame into Hugging Face Dataset
from datasets import Dataset

def format_example(row):
    return f"Type: {row['Type']}\nQuestion: {row['Question']}\nAnswer: {row['Answer']}"

df['formatted'] = df.apply(format_example, axis=1)

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df[['formatted']].rename(columns={"formatted": "text"}))

# Tokenize Dataset
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Split dataset
train_test = tokenized_dataset.train_test_split(test_size=0.2)
train_data = train_test["train"]
val_data = train_test["test"]


Map:   0%|          | 0/2354 [00:00<?, ? examples/s]

In [8]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Configure LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)

# Prepare model for LoRA training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

from trl import DataCollatorForCompletionOnlyLM, SFTConfig, SFTTrainer

# Define a response template to pass into DataCollatorForCompletionOnlyLM
response_template = "<|endoftext|>"

# Set up Data Collator with the response template
data_collator = DataCollatorForCompletionOnlyLM(
    tokenizer=tokenizer,
    response_template=response_template,
    return_tensors="pt"
)

# Trainer configuration with minimal logging by setting logging_steps to a high value
sft_config = SFTConfig(
    output_dir="./llama_custom_model",
    max_seq_length=512,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    save_steps=100,  # Save checkpoints only
    logging_steps=1000000,  # Set to a high number to avoid frequent logging
    evaluation_strategy="no",  # Skip evaluation during training
    learning_rate=1e-4,
    fp16=True,
    report_to="none"
)

# Initialize Trainer without tracking training loss
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_data,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Start training without loss tracking or logging
trainer.train()
trainer.save_model("./llama_custom_model")
print("Model training completed and saved.")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:187: UserWarning: Could not find response key `<|endoftext|>` in the following instance: <|begin_of_text|>Type: Structure
Question: In what chemical form do plants generally obtain nitrogen from soil?
Answer: Nitrate $ NO3 - <|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|

Step,Training Loss


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:187: UserWarning: Could not find response key `<|endoftext|>` in the following instance: <|begin_of_text|>Type: Essay
Question: Describe the upward movement of water and minerals in a plant.
Answer: The Upward Movement of Water and Minerals in Plants
The upward movement of water and minerals in plants, commonly referred to as the ascent of sap, is a critical physiological process that ensures the plant remains hydrated and nourished. This movement primarily occurs through the plant's vascular system, specifically within the xylem, and is facilitated by a mechanism known as bulk flow. The ascent of sap can be explained through the cohesion-tension hypothesis, which provides insights into how water travels from the roots to the leaves against the force of gravity.

According to the cohesion-tension hypothesis, transpiration plays a vital role in creating a pulling force that fa

Model training completed and saved.


In [9]:
# Save the model locally
OUTPUT_DIR = "./llama_custom_model"
trainer.save_model(OUTPUT_DIR)
print("Model saved locally at:", OUTPUT_DIR)


Model saved locally at: ./llama_custom_model


In [11]:
from transformers import pipeline

# Reload model for inference
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    return_full_text=False
)

# Define a sample prompt
sample_prompt = "Question: What is the structure of DNA?\nInformation:\n"
print("Prompt:", sample_prompt)

# Generate response
output = pipe(sample_prompt)
print("Generated Answer:", output[0]["generated_text"])


The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

Prompt: Question: What is the structure of DNA?
Information:

Generated Answer: A. Double helix
B. Single helix
C. Triple helix
D. Quadruple helix

Answer: A. Double helix
Explanation: DNA (Deoxyribonucleic acid) is a double-stranded helix, where two complementary strands are twisted together. Each strand is composed of nucleotides, which are linked by hydrogen bonds between the base pairs. The sugar-phosphate backbone provides the structural framework for the DNA molecule. The double helix structure of DNA was first proposed by James Watson and Francis Crick in 1953, based on X-ray crystallography data collected by
